### Import Libraries

In [2]:
import pandas as pd
import numpy as np

### Import Data

In [3]:
# Load datasets
fraud_data = pd.read_csv('../data/processed/fraud_data_cleaned.csv')
ip_country = pd.read_csv('../data/raw/IpAddress_to_Country.csv')

In [4]:
ip_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138846 entries, 0 to 138845
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   lower_bound_ip_address  138846 non-null  float64
 1   upper_bound_ip_address  138846 non-null  int64  
 2   country                 138846 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 3.2+ MB


In [ ]:
# Ensure IP addresses are integer for comparison
fraud_data['ip_address'] = fraud_data['ip_address'].astype(int)
ip_country['lower_bound_ip_address'] = ip_country['lower_bound_ip_address'].astype(int)
ip_country['upper_bound_ip_address'] = ip_country['upper_bound_ip_address'].astype(int)

### Geolocation Integration

#### Merge Fraud_Data.csv with IpAddress_to_Country.csv using range-based lookup

In [9]:
# Sort both dataframes by the IP columns (required for merge_asof)
fraud_data = fraud_data.sort_values('ip_address')
ip_country = ip_country.sort_values('lower_bound_ip_address')

# Use merge_asof to find the closest lower bound
fraud_data_merged = pd.merge_asof(
    fraud_data, 
    ip_country, 
    left_on='ip_address', 
    right_on='lower_bound_ip_address'
)

# Validate that the ip_address actually falls within the range [lower, upper]
# If it's outside the range, set the country to 'Unknown'
fraud_data_merged['country'] = np.where(
    (fraud_data_merged['ip_address'] >= fraud_data_merged['lower_bound_ip_address']) & 
    (fraud_data_merged['ip_address'] <= fraud_data_merged['upper_bound_ip_address']),
    fraud_data_merged['country'],
    'Unknown'
)

# Clean up temporary lookup columns
fraud_data_merged = fraud_data_merged.drop(['lower_bound_ip_address', 'upper_bound_ip_address'], axis=1)

# Save to processed folder
import os
fraud_data_merged.to_csv('../data/processed/fraud_data_with_country.csv', index=False)

#### Analyze Fraud Patterns by Country

In [10]:
# 1. Top 10 Countries by Total Transactions
print("Top 10 Countries by Volume:")
print(fraud_data_merged['country'].value_counts().head(10))

# 2. Top 10 Countries by Fraud Count
fraud_counts = fraud_data_merged[fraud_data_merged['class'] == 1]['country'].value_counts()
print("\nTop 10 Countries with most Fraudulent Transactions:")
print(fraud_counts.head(10))

# 3. Fraud Rate by Country (Percentage of transactions that are fraud)
country_stats = fraud_data_merged.groupby('country').agg(
    total_transactions=('class', 'count'),
    fraud_cases=('class', 'sum')
)
country_stats['fraud_rate'] = (country_stats['fraud_cases'] / country_stats['total_transactions']) * 100

# Filter for countries with a significant sample size (e.g., > 50 transactions)
high_risk_countries = country_stats[country_stats['total_transactions'] > 50].sort_values(by='fraud_rate', ascending=False)

print("\nCountries with Highest Fraud Rates (>50 transactions):")
print(high_risk_countries[['total_transactions', 'fraud_rate']].head(10))

Top 10 Countries by Volume:
country
United States        58049
Unknown              21966
China                12038
Japan                 7306
United Kingdom        4490
Korea Republic of     4162
Germany               3646
France                3161
Canada                2975
Brazil                2961
Name: count, dtype: int64

Top 10 Countries with most Fraudulent Transactions:
country
United States        5551
Unknown              1883
China                1043
Japan                 715
United Kingdom        477
Korea Republic of     380
Canada                348
France                300
Brazil                270
Germany               262
Name: count, dtype: int64

Countries with Highest Fraud Rates (>50 transactions):
              total_transactions  fraud_rate
country                                     
Luxembourg                    72   38.888889
Ecuador                      106   26.415094
Tunisia                      118   26.271186
Peru                         119   26.05

### Feature Engineering

#### Time Based Features

In [14]:
# Assuming 'purchase_time' and 'signup_time' are already datetime objects
# 1. Extract cyclic time features
fraud_df = fraud_data_merged
fraud_df['signup_time'] = pd.to_datetime(fraud_df['signup_time'])
fraud_df['purchase_time'] = pd.to_datetime(fraud_df['purchase_time'])
fraud_df['hour_of_day'] = fraud_df['purchase_time'].dt.hour
fraud_df['day_of_week'] = fraud_df['purchase_time'].dt.dayofweek

# 2. Time since signup (in seconds)
# Fraudsters often create an account and make a purchase immediately.
fraud_df['time_since_signup'] = (fraud_df['purchase_time'] - fraud_df['signup_time']).dt.total_seconds()

# Convert to hours or days if the numbers are too large for your model
fraud_df['time_since_signup_h'] = fraud_df['time_since_signup'] / 3600

#### Transaction frequency and velocity

In [18]:
# 1. Frequency per Device
# Count how many times each device_id appears in the entire dataset
fraud_df['device_id_count'] = fraud_df.groupby('device_id')['device_id'].transform('count')

# 2. Frequency per IP Address
# Similar to device, multiple users on one IP can indicate a botnet or proxy
fraud_df['ip_address_count'] = fraud_df.groupby('ip_address')['ip_address'].transform('count')

# 3. Rolling Window Velocity (Advanced)
# 1. Sort by user and time
fraud_df = fraud_df.sort_values(by=['user_id', 'purchase_time'])

# 2. Set purchase_time as index to allow time-based rolling windows
fraud_df = fraud_df.set_index('purchase_time')

# 3. Calculate velocity (transactions per user in the last 24 hours)
# We group by user_id and then use a 24-hour rolling window
fraud_df['user_transaction_velocity'] = (
    fraud_df.groupby('user_id')['user_id']
    .rolling('24H')
    .count()
    .reset_index(level=0, drop=True) # Removes the extra 'user_id' index level created by groupby
)

# 4. Reset index if you want purchase_time back as a column
fraud_df = fraud_df.reset_index()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10852\1124936853.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .rolling('24H')


In [19]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 1. Scale Numerical Features
# Features like 'time_since_signup' have huge ranges, while 'age' is small.
scaler = StandardScaler()
num_features = ['purchase_value', 'age', 'time_since_signup', 'device_id_count', 'ip_address_count']
fraud_df[num_features] = scaler.fit_transform(fraud_df[num_features])

# 2. One-Hot Encoding for Categorical Features
# Convert Browser, Source, and Sex into binary columns
fraud_df = pd.get_dummies(fraud_df, columns=['source', 'browser', 'sex'], drop_first=True)

In [20]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Define Features (X) and Target (y)
X = fraud_df.drop(['class', 'user_id', 'device_id', 'signup_time', 'purchase_time', 'ip_address', 'country'], axis=1)
y = fraud_df['class']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(f"Before SMOTE: {y_train.value_counts()}")
print(f"After SMOTE: {y_train_res.value_counts()}")

Before SMOTE: class
0    109568
1     11321
Name: count, dtype: int64
After SMOTE: class
0    109568
1    109568
Name: count, dtype: int64
